<center>

*******************************************************************************************
<font size="4">
Preprocessing of data from oscillating fingers under different emotional states <br>
</font>

##### 6 June 2025

##### Juan Ignacio Mendoza Garay

*******************************************************************************************

</center>

##### INFORMATION:

* This program preprocesses the raw motion-capture data collected for an experiment reported by  Smykovskyi et al., (2024).
>
* Reference:
    * Smykovskyi, A., Janaqi, S., Pla, S., Jean, P., Bieńkiewicz, M., & Bardy, B. G. (2024). Negative emotions disrupt intentional synchronization during emotion sensorimotor interaction. *Emotion, 24*(3), 687.
>
* Raw data retrieved on 22 May, 2025.:
    * https://osf.io/5qa4j/
    * https://supp.apa.org/psycarticles/supplemental/emo0001282/emo0001282_supp.html
>
* Instructions:

    Edit the values indicated with an arrow litest_mod_strengthe this: <---

*******************************************************************************************

In [3]:
import os
import sys

import pandas as pd

sys.path.append('C:/Users/me/SynCoord/src')
import syncoord as sc

***
### Declare paths and set parameters

In [1]:
main_data_dir = 'C:/Users/me/Oscillating_Fingers' # <--- parent directory for data

raw_data_dir = main_data_dir + '/raw_data' # <--- folder iwth raw data from repository
dinfo_fn = raw_data_dir + '/FINAL_ALL_TIS_and_TTS.csv' # <--- originally in folder "Analysis code"
trord_fn = raw_data_dir + '/trials_orders.csv' # <--- originally in supplemental materials docx file
raw_mocap_path = raw_data_dir + '/Behavioral modality' # <--- folder with raw mocap data files or file name

prep_data_dir = main_data_dir + '/prep_data' # <--- folder to save preprocessed data
annot_path = prep_data_dir + '/Oscillating_Fingers_annot.csv' # <--- file name for annotations (e.g., ID, Start, Sections, emotion)
prep_mocap_folder = prep_data_dir + '/prep_mocap' # <--- folder to save preprocessed mocap data files

***
### Pre-process data

>
Reformat mocap data files:

In [3]:
n_p = 3 # number of points
dim_lbl = ['x','y','z']
header = 5
usecols = range(2,11)
new_colnames = [f'{i_p}_{d_l}' for i_p in range(n_p) for d_l in dim_lbl]
raw_mocap_path_is_file = os.path.isfile(raw_mocap_path)
if raw_mocap_path_is_file: fn_list = [raw_mocap_path]
else: fn_list = sc.utils.listfiles(raw_mocap_path)
for fn in fn_list:
    ffn_in = os.path.join(raw_mocap_path, fn)
    mcd = pd.read_csv(ffn_in, header=header, usecols=usecols, names=new_colnames)
    # Fill missing data:
    idx_nan = mcd.index[ mcd.isna().any(axis=1)]
    len_idx_nan = len(idx_nan)
    if len_idx_nan:
        mcd = mcd.interpolate(limit_direction='both',method='cubicspline')
        warning_interp = f'{len_idx_nan} missing raw data points have been interpolated'
        print('Warning:',warning_interp)
    fn_ne = os.path.splitext(os.path.basename(fn))[0]
    ffn_out = os.path.join(prep_mocap_folder, fn_ne+'.parquet')
    mcd.to_parquet(ffn_out)
mcd.head(3)

,0_x,0_y,0_z,1_x,1_y,1_z,2_x,2_y,2_z
0,0.021956,0.032296,0.131360,0.189366,0.084815,0.053371,0.192847,0.064292,0.281151
1,0.020728,0.029514,0.131017,0.189620,0.084550,0.053335,0.192812,0.063358,0.281860
2,0.020563,0.029139,0.130957,0.189890,0.084145,0.053303,0.192656,0.062615,0.282326


>
Make annotations file:

In [5]:
verbose = False
annots_colnames = ['ID','Name','emotion','triad','trial']
rename_emotions = {'Neg':'Negative','Pos':'Positive','Netr':'Neutral'}
main_name = 'Oscillating Fingers'
dinfo_df = pd.read_csv(dinfo_fn)
trord_df = pd.read_csv(trord_fn)
prep_mocap_fnames = sc.utils.listfiles(prep_mocap_folder)
annots = []
for fn in prep_mocap_fnames:
    fn_ne = os.path.splitext(fn)[0]
    trtr_str = fn_ne.replace('triad','').replace('trial','')
    i_triad_str, i_trial_str = trtr_str.split('_')
    i_triad, i_trial = int(i_triad_str), int(i_trial_str)
    i_order = dinfo_df.Order[dinfo_df.Triad==i_triad ].values[0]
    raw_emo_str = trord_df[f'Order {i_order}'][trord_df['Trial #'] == i_trial].values[0]
    new_emo_str = rename_emotions[raw_emo_str]
    if verbose:
        print('i_triad =',i_triad,'\ni_order =',i_order,'\ni_trial =',i_trial)
        print('emo_str [raw,new] =',[raw_emo_str,new_emo_str],'\n')
    # break
    ID = fn_ne
    Name = f'{main_name} - triad{i_triad_str}, trial{i_trial_str}, {new_emo_str}'
    annots.append([ID,Name,new_emo_str,i_triad,i_trial])
annots_df = pd.DataFrame(annots, columns=annots_colnames)
annots_df.to_csv(annot_path, index=False)
annots_df.head(3)

,ID,Name,emotion,triad,trial
0,triad10_trial1,"Oscillating Fingers - triad10, trial1, Negative",Negative,10,1
1,triad10_trial10,"Oscillating Fingers - triad10, trial10, Positive",Positive,10,10
2,triad10_trial11,"Oscillating Fingers - triad10, trial11, Negative",Negative,10,11
